# Backend

In [8]:
get_ipython().system('pip install voila')
get_ipython().system('pip install yfinance')
get_ipython().system('pip install PyPortfolioOpt')
get_ipython().system('pip install cvxopt')
get_ipython().system('pip install cvxpy')
get_ipython().system('pip install matplotlib')
get_ipython().system('pip install sklearn')
import pypfopt
import yfinance as yf
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from pypfopt import risk_models
from pypfopt import plotting
from pypfopt import expected_returns
from pypfopt import EfficientFrontier
from pypfopt import DiscreteAllocation
from pypfopt import objective_functions
from pypfopt import DiscreteAllocation
get_ipython().system('pip install qiskit')
get_ipython().run_line_magic('matplotlib', 'inline')
from qiskit.finance import QiskitFinanceError
from qiskit.finance.data_providers import *
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
import datetime
import matplotlib.pyplot as plt
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

from qiskit import Aer
from qiskit.circuit.library import TwoLocal
from qiskit.aqua import QuantumInstance
from qiskit.finance.applications.ising import portfolio
from qiskit.optimization.applications.ising.common import sample_most_likely
from qiskit.finance.data_providers import RandomDataProvider
from qiskit.aqua.algorithms import VQE, QAOA, NumPyMinimumEigensolver
from qiskit.aqua.components.optimizers import COBYLA
import numpy as np
import matplotlib.pyplot as plt
import datetime

/srv/conda/envs/notebook/lib/python3.7/site-packages/qiskit/finance/__init__.py:50: DeprecationWarning: The package qiskit.finance is deprecated. It was moved/refactored to qiskit_finance (pip install qiskit-finance). For more information see <https://github.com/Qiskit/qiskit-aqua/blob/master/README.md#migration-guide>
  warn_package('finance', 'qiskit_finance', 'qiskit-finance')


In [5]:
def returns(prices):
    mu = expected_returns.capm_return(prices)
    return mu

def min_variance(prices):
    # Long/Short Min Variance Portfolio
    S = risk_models.CovarianceShrinkage(prices).ledoit_wolf()
    ef = EfficientFrontier(None, S, weight_bounds=(None, None))
    ef.min_volatility()
    weights = ef.clean_weights()
    ef.portfolio_performance(verbose=True);
    return(weights)

def Max_Sharpe_port(prices):
    # Max Sharpe Portfolio
    mu = returns(prices)
    S = risk_models.CovarianceShrinkage(prices).ledoit_wolf()
    ef = EfficientFrontier(mu, S) 
    ef.max_sharpe()
    weights = ef.clean_weights()
    ef.portfolio_performance(verbose=True)
    latest_prices = prices.iloc[-1]  # prices as of the day you are allocating
    da = DiscreteAllocation(weights, latest_prices, total_portfolio_value=100000, short_ratio=0.3)
    alloc, leftover = da.lp_portfolio()
    print(f"Discrete allocation performed with ${leftover:.2f} leftover")
    print(alloc)
    return alloc

risk = 0.15
def return_given_risk(prices):
    mu = returns(prices)
    S = risk_models.CovarianceShrinkage(prices).ledoit_wolf()
    ef = EfficientFrontier(mu, S)
    ef.add_objective(objective_functions.L2_reg)  # gamme is the tuning parameter
    ef.efficient_risk(0.15)
    weights = ef.clean_weights()
    ef.portfolio_performance(verbose=True)
    return weights

def minimize_risk(prices):
    mu = returns(prices)
    S = risk_models.CovarianceShrinkage(prices).ledoit_wolf()
    ef = EfficientFrontier(mu, S, weight_bounds=(None, None))
    ef.add_objective(objective_functions.L2_reg)
    ef.efficient_return(target_return=0.07, market_neutral=True)
    weights = ef.clean_weights()
    ef.portfolio_performance(verbose=True);
    return weights

def generate_prices(tickers):
    tickers = tickers 
    ohlc = yf.download(tickers, end="2020-12-23")
    prices = ohlc["Adj Close"].dropna(how="all")
    return(prices)

def index_to_selection(i, num_assets):
    s = "{0:b}".format(i).rjust(num_assets)
    x = np.array([1 if s[i]=='1' else 0 for i in reversed(range(num_assets))])
    return x

def print_result(result):
    selection = sample_most_likely(result.eigenstate)
    value = portfolio.portfolio_value(selection, mu, sigma, q, budget, penalty)
    print('Optimal: selection {}, value {:.4f}'.format(selection, value))

    eigenvector = result.eigenstate if isinstance(result.eigenstate, np.ndarray) else result.eigenstate.to_matrix()
    probabilities = np.abs(eigenvector)**2
    i_sorted = reversed(np.argsort(probabilities))
    print('\n----------------- Full result ---------------------')
    print('selection\tvalue\t\tprobability')
    print('---------------------------------------------------')
    for i in i_sorted:
        x = index_to_selection(i, num_assets)
        value = portfolio.portfolio_value(x, mu, sigma, q, budget, penalty)
        probability = probabilities[i]
        print('%10s\t%.4f\t\t%.4f' %(x, value, probability))

# Discrete Allocation
# latest_prices = prices.iloc[-1]  # prices as of the day you are allocating
# da = DiscreteAllocation(weights, latest_prices, total_portfolio_value=20000, short_ratio=0.3)
# alloc, leftover = da.lp_portfolio()
# print(f"Discrete allocation performed with ${leftover:.2f} leftover")
# alloc

# VQE Backend - Shitty Results , erratic convergence pattern
# backend = Aer.get_backend('statevector_simulator')
# seed = 50

# cobyla = COBYLA()
# cobyla.set_options(maxiter=500)
# ry = TwoLocal(qubitOp.num_qubits, 'ry', 'cz', reps=3, entanglement='full')
# vqe = VQE(qubitOp, ry, cobyla)
# vqe.random_seed = seed

# quantum_instance = QuantumInstance(backend=backend, seed_simulator=seed, seed_transpiler=seed)

# result = vqe.run(quantum_instance)

# print_result(result)

def quantum(tickers):
    data = YahooDataProvider(
                     tickers = tickers,
#                      start=datetime.datetime(2015,3,23),
                     end=datetime.datetime(2020, 3, 23))
    data.run()

    num_assets = 5

    mu = data.get_period_return_mean_vector()
    sigma = data.get_period_return_covariance_matrix()

    q = 0.5                   # set risk factor
    budget = num_assets // 2  # set budget
    penalty = num_assets      # set parameter to scale the budget penalty term

    qubitOp, offset = portfolio.get_operator(mu, sigma, q, budget, penalty)

    backend = Aer.get_backend('statevector_simulator')
    seed = 50

    cobyla = COBYLA()
    cobyla.set_options(maxiter=250)
    qaoa = QAOA(qubitOp, cobyla, 3)

    qaoa.random_seed = seed

    quantum_instance = QuantumInstance(backend=backend, seed_simulator=seed, seed_transpiler=seed)

    result = qaoa.run(quantum_instance)

    print_result(result)

# Front End - UI

In [6]:
import warnings
warnings.filterwarnings('ignore')

import ipywidgets as widgets
from IPython.display import display, clear_output

!jupyter nbextension enable --py widgetsnbextension --sys-prefix
!jupyter serverextension enable voila --sys-prefix

# file = open("logo.png", "rb")
# image = file.read()
# image_headline = widgets.Image(
#                     value=image,
#                     format='jpg',
#                     width='200'
# )
# vbox_logo = widgets.VBox([image_headline])

text0 = widgets.HTML(value="<h2>How much would you like to invest?</h2>")
investment = widgets.BoundedIntText(
    value=0,
    min=0,
    max=10000000,
    step=1,
    description='Investment:',
    disabled=False
)
vbox_invest = widgets.VBox([text0, investment])


text1 = widgets.HTML(value="<h2>Please select industries you want to invest in.</h2>")
iTech = widgets.Checkbox(
    value=False,
    description='Information Technology',
    disabled=False,
    indent=False
)
hCare = widgets.Checkbox(
    value=False,
    description='Health Care',
    disabled=False,
    indent=False
)
fin = widgets.Checkbox(
    value=False,
    description='Financials',
    disabled=False,
    indent=False
)
cDis = widgets.Checkbox(
    value=False,
    description='Consumer Discretionary',
    disabled=False,
    indent=False
)
cSer = widgets.Checkbox(
    value=False,
    description='Consumer Services',
    disabled=False,
    indent=False
)
ind = widgets.Checkbox(
    value=False,
    description='Industrials',
    disabled=False,
    indent=False
)
cStap = widgets.Checkbox(
    value=False,
    description='Consumer Staples',
    disabled=False,
    indent=False
)
eng = widgets.Checkbox(
    value=False,
    description='Energy',
    disabled=False,
    indent=False
)
util = widgets.Checkbox(
    value=False,
    description='Utilities',
    disabled=False,
    indent=False
)
rEst = widgets.Checkbox(
    value=False,
    description='Real Estate',
    disabled=False,
    indent=False
)
mat = widgets.Checkbox(
    value=False,
    description='Materials',
    disabled=False,
    indent=False
)
All = widgets.Checkbox(
    value=False,
    description='All',
    disabled=False,
    indent=False
)
vbox_industries = widgets.VBox([text1, iTech, hCare, fin, cDis, cSer, ind, cStap, eng, util, rEst, mat, All])

text2 = widgets.HTML(value="<h2>What level of risk are you comfortable with?</h2>")
risk = widgets.ToggleButtons(
        options=['Low', 'Medium', 'High']
)
vbox_risk = widgets.VBox([text2, risk])

page = widgets.VBox([vbox_invest, vbox_industries, vbox_risk]) #vbox_logo
display(page)

#Submit button outputs values captured in above widgets
submit = widgets.Button(description="Submit")
output = widgets.Output()

display(submit, output)

def on_button_clicked(b):
    with output:
        clear_output()
        print(f"Investment amount: {investment.value}")
        print(f"Selected industries: {iTech.value}, {hCare.value}, {fin.value}, {cDis.value}, {cSer.value}, {ind.value}, {cStap.value}, {eng.value}, {util.value}, {rEst.value}, {mat.value}, {All.value}")
        print(f"Risk level: {risk.value}")

submit.on_click(on_button_clicked)
vbox_output = widgets.VBox([submit, output])

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK
Enabling: voila
- Writing config: /srv/conda/envs/notebook/etc/jupyter
    - Validating...
      voila 0.2.9 OK


Button(description='Submit', style=ButtonStyle())

Output()

In [9]:
industries_all = (iTech, hCare, fin, cDis, cSer, ind, cStap, eng, util, rEst, mat, All)
sectors = []

for i in industries_all:
    if i.value == True:
        sectors.append(i.description)
    
test1 = ["MSFT", "AMZN", "KO", "MA", "COST"] # Tech / Consumer 
q1 = ['MSFT','AMZN']

test2 = ["LUV", "XOM", "PFE", "JPM", "UNH"] # Pharma / Financial / Healthcare
q2 = ['PFE', 'UNH', "XOM"]

test3 = ["ACN", "DIS", "GILD", "F", "TSLA"] # Transport / Consulting 
q3 = ['ACN', 'TSLA']

# print(investment.value)
# print(sectors)
# print(risk.value)

def compute(investment, sectors, risk):
    if risk.value == "Low":
        print("Low Risk - Long Portfolio in 'good' stocks in this industry \n")
        prices = generate_prices(test1)
        cov = risk_models.CovarianceShrinkage(prices).ledoit_wolf()
        min_variance(prices)
        print(f"Equal distribition in {test1}" )
        return(test1)
    if risk.value == "Medium":
        print("Medium Risk - Optimal Risk/Return ratio calculated by Markowitz Model \n")
        prices = generate_prices(test1)
        cov = risk_models.CovarianceShrinkage(prices).ledoit_wolf()
        return(Max_Sharpe_port(prices))
    if risk.value == "High":
        print("High Risk - Quantum return model powered by our proprietary algorithm run on IBMQ \n")
        prices = generate_prices(q1)
        risk_models.CovarianceShrinkage(prices).ledoit_wolf()
        return(Max_Sharpe_port(prices))

    
trade = compute(investment, sectors, risk)

Medium Risk - Optimal Risk/Return ratio calculated by Markowitz Model 

[*********************100%***********************]  5 of 5 completed
Expected annual return: 24.2%
Annual volatility: 15.4%
Sharpe Ratio: 1.44
Discrete allocation performed with $4.66 leftover
{'AMZN': 7, 'COST': 33, 'KO': 218, 'MA': 140, 'MSFT': 35}


# Trade Execution

In [4]:
get_ipython().system('pip install alpaca-trade-api')

import alpaca_trade_api as tradeapi
# from alpaca_trade_api.rest import TimeFrame
# import pandas as pd

APCA_API_KEY_ID = 'PKFKM6KPI1COLEQEMB7Y'
APCA_API_SECRET_KEY = 'GWDe2JErqsrB3A6ScplTdaWjDLelqKNl5Gmz3zcW'
APCA_API_BASE_URL = 'https://paper-api.alpaca.markets'

# Setting up API and account details
api = tradeapi.REST(
    APCA_API_KEY_ID,
    APCA_API_SECRET_KEY,
    base_url=APCA_API_BASE_URL,
    api_version='v2')

def market_order_from_list(orderList):
    """
    Creates live market buy orders which expire at end of the day from a list of tuples in the form:
    orderList = [('AAPL', '10'), ('JPM', '5')]
    """
    for ticker, amount in orderList:
        api.submit_order(
            symbol=f'{ticker}',
            side='buy',
            type='market',
            qty=f'{amount}',
            time_in_force='day'
        )

     |████████████████████████████████| 61 kB 3.7 MB/s eta 0:00:011
     |████████████████████████████████| 273 kB 8.1 MB/s eta 0:00:01


In [98]:
trade

{'AMZN': 18, 'MSFT': 189}

In [99]:
orderList = [('AMZN', 18),
             ('MSFT', 189)]


In [11]:
market_order_from_list(orderList)

api.list_orders()


NameError: name 'market_order_from_list' is not defined

In [101]:
api.cancel_all_orders()